# Measuring Data Quality

## Measures of Data Quality

- Validity: Conforms to a Schema
- Accuracy: Conforms to gold standard
- Completeness: All records?
- Consistency: Matches other data
- Uniformity: Same units

## Blueprint for Cleaning Data

- Audit your Data
- Create a Data Cleaning Plan
  - Identify causes
  - Define operations
  - Test
- Execute the plan (run a script)
- Manually correct


Iterate on the process

# Quiz 1

In [35]:
"""
Your task is to check the "productionStartYear" of the DBPedia autos datafile for valid values.
The following things should be done:
- check if the field "productionStartYear" contains a year
- check if the year is in range 1886-2014
- convert the value of the field to be just a year (not full datetime)
- the rest of the fields and values should stay the same
- if the value of the field is a valid year in the range as described above,
  write that line to the output_good file
- if the value of the field is not a valid year as described above, 
  write that line to the output_bad file
- discard rows (neither write to good nor bad) if the URI is not from dbpedia.org
- you should use the provided way of reading and writing data (DictReader and DictWriter)
  They will take care of dealing with the header.

You can write helper functions for checking the data and writing the files, but we will call only the 
'process_file' with 3 arguments (inputfile, output_good, output_bad).
"""
import csv
import pprint
import datetime as dt

INPUT_FILE = 'data/autos.csv'
OUTPUT_GOOD = 'data/autos-valid.csv'
OUTPUT_BAD = 'data/FIXME-autos.csv'

In [38]:
def get_date(date_str):
    try:
        dt_string = date_str[:-5] + date_str[-5:].replace(':', '')
        return dt.datetime.strptime(dt_string, '%Y-%m-%dT%H:%M:%S%z')
    except ValueError:
        return None

In [39]:
def date_in_range(date):
    min_year = 1886
    max_year = 2014
    return (min_year <= date.year) and (date.year <= max_year)

In [40]:
input_file = INPUT_FILE
output_good = OUTPUT_GOOD
output_bad = OUTPUT_BAD

good_data = list()
bad_data = list()

with open(input_file, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames
        start_year_idx = header.index('productionStartYear')
        for row in reader:
            # Check that it is a date
            date = get_date(row['productionStartYear'])
            if not date:
                bad_data.append(row)
                continue
            # Check the year's range
            if not date_in_range(date):
                bad_data.append(row)
                continue
            

True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [14]:
header

['URI',
 'rdf-schema#label',
 'rdf-schema#comment',
 'assembly_label',
 'assembly',
 'automobilePlatform_label',
 'automobilePlatform',
 'bodyStyle_label',
 'bodyStyle',
 'class_label',
 'class',
 'designCompany_label',
 'designCompany',
 'designer_label',
 'designer',
 'engine_label',
 'engine',
 'fuelCapacity',
 'height',
 'layout_label',
 'layout',
 'length',
 'manufacturer_label',
 'manufacturer',
 'modelEndYear',
 'modelStartYear',
 'parentCompany_label',
 'parentCompany',
 'predecessor_label',
 'predecessor',
 'productionEndDate',
 'productionEndYear',
 'productionStartDate',
 'productionStartYear',
 'relatedMeanOfTransportation_label',
 'relatedMeanOfTransportation',
 'sales_label',
 'sales',
 'successor_label',
 'successor',
 'thumbnail_label',
 'thumbnail',
 'transmission',
 'variantOf_label',
 'variantOf',
 'vehicle_label',
 'vehicle',
 'weight',
 'wheelbase',
 'width',
 'point',
 '22-rdf-syntax-ns#type_label',
 '22-rdf-syntax-ns#type',
 'wgs84_pos#lat',
 'wgs84_pos#long',
 '

In [6]:
def process_file(input_file, output_good, output_bad):

    with open(input_file, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames

        #COMPLETE THIS FUNCTION



    # This is just an example on how you can use csv.DictWriter
    # Remember that you have to output 2 files
    with open(output_good, "w") as g:
        writer = csv.DictWriter(g, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in YOURDATA:
            writer.writerow(row)

In [7]:
def test():
    process_file(INPUT_FILE, OUTPUT_GOOD, OUTPUT_BAD)


test()

NameError: name 'YOURDATA' is not defined